In [1]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [2]:
#导入数据

dorm_feature=pd.read_csv('dorm_input/dorm_test.csv')
dorm_feature.head()

,id,dorm_date,out_or_in,date,year,month,hour,weekday,dorm_hour00,dorm_hour01,...,dorm_hour23,dorm_hour_01_05,dorm_hour_morning,dorm_hour_afternoon,dorm_hour_evening,dorm_hour_midnight,dorm_resting_day,dorm_working_day,dorm_on_school,dorm_vacation
0,28358,2014-01-21 05:58:02,1,01-21-2014,2014,v,a,wd,0,0,...,0,1,0,0,0,0,0,1,0,1
1,14167,2014-01-21 07:08:13,1,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1
2,14167,2014-01-21 07:14:19,0,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1
3,8280,2014-01-21 08:33:22,0,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1
4,1511,2014-01-21 09:30:28,1,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1


In [3]:
#导入数据

dorm_feature=pd.read_csv('dorm_input/dorm_test_2.csv')
dorm_feature.head()

,id,dorm_date,out_or_in,date,year,month,hour,weekday,dorm_hour00,dorm_hour01,...,dorm_hour23,dorm_hour_01_05,dorm_hour_morning,dorm_hour_afternoon,dorm_hour_evening,dorm_hour_midnight,dorm_resting_day,dorm_working_day,dorm_on_school,dorm_vacation
0,19326,2014-01-21 06:12:13,0,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1
1,15190,2014-01-21 06:49:12,1,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1
2,18610,2014-01-21 06:51:00,0,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1
3,462,2014-01-21 07:40:51,1,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1
4,18610,2014-01-21 07:51:06,1,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1


In [4]:
#计算进出寝室的总次数

dorm_feature_f=DataFrame(dorm_feature.groupby('id')['id'].count())
dorm_feature_f.columns=['dorm_count_sum']

In [5]:
#计算活跃的天数

dorm_day=DataFrame(dorm_feature.groupby(['id','date'])['date'].count())
dorm_day.to_csv('dorm_input/dorm_day.csv',index=True)
dorm_day_count=pd.read_csv('dorm_input/dorm_day.csv')
dorm_day_count=DataFrame(dorm_day_count.groupby('id')['date.1'].count())
dorm_day_count.columns=['dorm_daycount']

In [6]:
#连接表

dorm_feature_f=dorm_feature_f.join(dorm_day_count)

In [7]:
#根据id聚合求解各个特征的产生的总数

count_columns=list(dorm_feature.columns)[8:]

#count_columns
count_pivot=dorm_feature.pivot_table(index='id',values=count_columns,aggfunc=np.count_nonzero)
dorm_feature_f=dorm_feature_f.join(count_pivot)

In [8]:
#根据id聚合求解各个特征占比(各特征进出总次数占比)

#a=['id','date','year']
mean_columns=[x for x in list(dorm_feature.columns)[8:]]
mean_pivot=dorm_feature.pivot_table(index='id',values=mean_columns,aggfunc=np.mean)

#对列名重命名
mean_pivot.columns=[x+'_rate' for x in list(mean_pivot.columns)]
dorm_feature_f=dorm_feature_f.join(mean_pivot)

In [9]:
#定义id关于每一列的平均进出次数(每天)

def get_colunms_mean(df_sum,df_count):
    a=list(df_count.columns)
    columns_mean=DataFrame(df_count[a[0]])#初始化表格大小
    columns_mean.columns=['id_count']
    for x in range(len(a)):
        columns_count_x=df_count[a[x]].values
        columns_sum=df_sum['dorm_daycount'].values
        columns_mean[a[x]+'_day_mean']=np.divide.reduce([columns_count_x,columns_sum],axis=0)
        x+=1
    columns_mean.drop('id_count',axis=1,inplace=True)
    return columns_mean

#运用定义的函数
dorm_columns_mean=get_colunms_mean(dorm_feature_f,count_pivot)

#连接总表
dorm_feature_f=dorm_feature_f.join(dorm_columns_mean)

In [10]:
#平均每天进出的次数

dorm_feature_f['dorm_day_mean']=dorm_feature_f['dorm_count_sum']/dorm_feature_f['dorm_daycount']

In [13]:
#导出数据

dorm_feature_f.to_csv('dorm_input/dorm_test_feature_f.csv',index='id')

In [12]:
#导出数据

dorm_feature_f.to_csv('dorm_input/dorm_test_feature_f_2.csv',index='id')

In [11]:
dorm_feature_f.head()

,dorm_count_sum,dorm_daycount,dorm_hour00,dorm_hour01,dorm_hour02,dorm_hour03,dorm_hour04,dorm_hour05,dorm_hour06,dorm_hour07,...,dorm_hour_01_05_day_mean,dorm_hour_afternoon_day_mean,dorm_hour_evening_day_mean,dorm_hour_midnight_day_mean,dorm_hour_morning_day_mean,dorm_on_school_day_mean,dorm_resting_day_day_mean,dorm_vacation_day_mean,dorm_working_day_day_mean,dorm_day_mean
id,,,,,,,,,,,,,,,,,,,,,
2,198,72,0,0,0,0,0,0,0,0,...,0,0,1,0,0,2,0,0,2,2.750000
32,179,71,0,0,0,0,0,0,1,0,...,0,1,0,0,0,2,0,0,2,2.521127
33,21,11,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1.909091
34,216,85,0,0,0,0,0,0,16,14,...,0,0,0,0,0,2,0,0,1,2.541176
35,416,88,0,0,0,0,0,0,10,70,...,0,1,0,0,2,4,1,0,3,4.727273
